In [1]:
from db1_preprocess_utils import *
from feature_extraction import *
from experiment_one_utils import *

In [2]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from scipy.signal import stft

In [3]:
path = 'D:\\AI-Workspace\\sEMGClassification\\GestureClassificationUsingCViT\\data\\DB1\\raw'

signal_type='raw'
input_type='tkeo'
n_channels=64
low_cut=20
high_cut=500
order=6 
window_size=50
overlap=50
fs=2048

whiten=False
center=False
extend=False
extend_size=0
normalize=False
mu=0

start_subject = 6
total_subject = 20
session=[1,2]
ratio = 0.2

noise_db = [5,10,15]
std = 1
stochastic_depth_rate = 0.1

In [4]:
data, label, X_test, y_test = get_experiment_data(path, subjects=start_subject, sessions=session,
                                                       signal_type='raw', input_type=input_type, 
                                                       channels=n_channels, low_cut=20, high_cut=500,
                                                      order=6, window_size=window_size, overlap=overlap, fs=fs, 
                                                      extend=extend, center=center, 
                                                      extend_size=extend_size, whiten=whiten,
                                                      normalize=False, mu=0, ratio=0.2)

####################################################################################
Loading subject 6
####################################################################################
Size of the input data is (17021, 64, 100, 1)
The input label shape is (17021,)
The total number of classes is 34
************************************************************************************
Loaded TKEO input data
************************************************************************************


In [5]:
data.shape, label.shape, X_test.shape, y_test.shape

((17021, 64, 100, 1), (17021,), (4256, 64, 102), (4256,))

In [6]:
#Hyper-parameter

input_shape = data.shape[1:]
patch_size = 8
n_patches = 24 
dims = n_patches + 1 
n_transformer_layers = 2
n_heads = 2
transformer_units = [dims * 2,dims]  
mlp_head_units = [128, 34]   
n_classes = len(np.unique(label))
stochastic_depth = False

stochastic_depth_rate = 0.1

n_batches = 32
n_epochs = 150
n_folds = 3
type_of_experiment = 1

In [7]:
model = vit_gesture_classification(input_shape, dims, patch_size, n_patches,
                                  n_transformer_layers, n_heads, transformer_units,
                                   mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate)

model.summary()

Model: "sEMG-Decomposition"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 100, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 covolution_layer (Sequential)  (None, 32, 50, 25)   20413       ['input_1[0][0]']                
                                                                                                  
 generate__patches (Generate_Pa  (None, None, 1600)  0           ['covolution_layer[0][0]']       
 tches)                                                                                           
                                                                                 

run_experiment(n_folds, n_batches, n_epochs, start_subject, total_subject, session, path, 
                       input_type,n_channels, window_size,extend, extend_size, center, whiten, ratio, noise_db, std,
                      input_shape, dims, patch_size, n_patches,
                       n_transformer_layers, n_heads, transformer_units,
                       mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate, type_of_experiment)